In [1]:
import csv
import re
from estnltk import Text
from estnltk.resolve_layer_dag import make_resolver
from collections import Counter
import pandas as pd
from tqdm import tqdm

In [2]:
rows = []
with open("other_diag_text_extras_with_codegroups.csv", "r") as fin:
    reader = csv.reader(fin)
    for row in reader:
        rows.append(row)

1) Korja välja lühendid

Oc utr

Oc. utr.

saad sa nad kokku koguda. Sul on praegu punktid ja mittepunktiga lühendid eraldi

2) tõsta eraldi kõik tekstid milles on rohkem kui neli sõna

3) tõsta välja riskiastest sisaldavad tekstid

4) NYHA xxx väljendid eraldi

5) tõsta välja tekstid kus on sees sõna kasutab

6) tõsta välja tektid sõna olemasolu 

7) tõsta välja tekstid kus on sees rooma numbrid

8) tõsta välja tektid kus on sees araabia number või number ja  täht (1b)

9) tõsta välja tekstid kus on üks sõna suurte tähtedega

10) kas sa saad eemaldada kordused "Sin.
sin.", jne

11) tõsta välja tekstid sõnaga soovitav

In [3]:
def find_abbreviations_and_unks(raw_text):
    
    resolver = make_resolver(
                 disambiguate=False,
                 guess=False,
                 propername=False)
    
    abbrs = []
    unks = []
    words = []
    lemmas = []
    text = Text(raw_text)
    text.tag_layer(resolver=resolver)

    for idx, word in enumerate(text.morph_analysis):
        
        if 'Y' in word.morph_analysis.partofspeech:
            abbrs.append(text.words[idx].text)
        
        elif None in word.morph_analysis.lemma:
            possible_abbr = text.words[idx].text
            
            if re.search('[A-ZÜÕÖÄŽŠa-züõäöšž]', possible_abbr):
                if len(possible_abbr) < 4:
                    abbrs.append(possible_abbr)
                elif not re.search('[aeiou]', possible_abbr) :
                    abbrs.append(possible_abbr)
                elif not re.search('[a-züõäöšž]', possible_abbr):
                    abbrs.append(possible_abbr)
                else:
                    unks.append(possible_abbr)
        else:
            words.append(text.words[idx].text)
            lemmas.append(word.morph_analysis.lemma[0])
    return abbrs, unks, words, lemmas                

In [4]:
soovitav, repetition, capital, arabic, roman, olemasolu, kasutab, nyha, risk = [], [], [], [], [], [], [], [], [], 
longtext = []
abbreviations, unknowns, words, lemmas = [], [], [], []

s, r, c, a, ro, o, k, n, ri, l, ab = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

other_other = []

for row in tqdm(rows):
    
    # lühendite, sõnade sagedused
    abbrs, unks, w, lem = find_abbreviations_and_unks(row[0])
    abbreviations += abbrs
    unknowns += unks
    words += w
    lemmas += lem
    if len(abbrs) > 0:
        ab = 1
    
    split_text = row[0].lower().split()
    split_text_cap = row[0].split()
    split_text2 = []
    
    # kordused
    for idx, split in enumerate(split_text):
        if idx > 0 and split.strip(' .,') == split_text[idx-1].strip(' .,'):
            repetition.append(row)
            r = 1
            break
    
    # suurtähtedega sõnu sisaldavad
    for split in split_text_cap:
        if split.isupper() and len(split) > 2:
            if row not in capital:
                if not re.search('(^|\s|[-,])(?=[MDCLXVI])M*(C[MD]|D?C*)(X[CL]|L?X*)(I[XV]|V?I*)($|\s|[-,])', row[0]):
                    if 'NYHA' not in row[0]:
                        capital.append(row)
                        c = 1
                        break
    
    # numbrid
    if re.search('[0-9]', row[0]):
        arabic.append(row)
        a = 1
    
    # rooma numbrid
    if re.search('(^|\s|[-,])(?=[MDCLXVI])M*(C[MD]|D?C*)(X[CL]|L?X*)(I[XV]|V?I*)($|\s|[-,])', row[0]):
        roman.append(row)
        ro = 1
    
    if 'olemasolu' in row[0].lower():
        olemasolu.append(row)
        o = 1
        
    if 'kasuta' in row[0].lower():
        kasutab.append(row)
        k = 1
        
    if 'soovitav' in row[0].lower() or 'soovitatav' in row[0].lower():
        soovitav.append(row)
        s = 1    
    
    if 'NYHA' in row[0]:
        nyha.append(row)
        n = 1
        
    if 'risk' in row[0].lower():
        risk.append(row)
        ri = 1
    
    if len(split_text) > 4:
        longtext.append(row)
        l = 1

    if s == 0 and r == 0 and c == 0 and a == 0 and ro == 0 and o == 0 and k == 0 and n == 0 and ri == 0 and l == 0 and ab == 0:
        other_other.append(row)
 
    s, r, c, a, ro, o, k, n, ri, l, ab = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

100%|██████████| 13199/13199 [32:26<00:00,  6.78it/s]


In [5]:
Counter(abbreviations).most_common(50)

[('e', 626),
 ('dex', 603),
 ('sin', 597),
 ('II', 505),
 ('I', 292),
 ('in', 238),
 ('op', 206),
 ('st.', 175),
 ('Od', 160),
 ('os', 154),
 ('L5', 152),
 ('Os', 146),
 ('od', 144),
 ('a.', 142),
 ('NYHA', 136),
 ('s', 131),
 ('HAS-BLED', 128),
 ('ou', 124),
 ('E', 118),
 ('oc', 116),
 ('tür', 109),
 ('BMI', 103),
 ('t', 100),
 ('S1', 99),
 ('L4', 97),
 ('utr', 96),
 ('st', 94),
 ('h', 94),
 ('A', 91),
 ('v', 87),
 ('art', 87),
 ('l', 78),
 ('a', 71),
 ('M0', 71),
 ('Oc', 70),
 ('iit', 68),
 ('KMI', 67),
 ('CHA2DS2-VASc', 65),
 ('PR', 63),
 ('v.a', 62),
 ('chr', 62),
 ('Fr', 62),
 ('vas', 59),
 ('pos', 58),
 ('D', 57),
 ('Op', 57),
 ('neg', 55),
 ('Ou', 53),
 ('PTKA', 53),
 ('OS', 51)]

In [6]:
pd.DataFrame(Counter(abbreviations).most_common()).to_csv("temp/abbreviations.csv")

In [7]:
Counter(lemmas).most_common(50)

[('-', 1252),
 ('ja', 1044),
 ('parem', 903),
 ('vasak', 727),
 ('muu', 669),
 ('täpsustama', 455),
 ('hüpertooniatõbi', 371),
 ('südamepuudulikkus', 369),
 ('südamekahjustus', 362),
 ('III', 353),
 ('2', 319),
 ('aste', 301),
 ('3', 269),
 ('kahtlus', 252),
 ('või', 236),
 ('tegevus', 235),
 ('risk', 232),
 ('ilm', 228),
 ('krooniline', 213),
 ('kasvaja', 203),
 ('1', 200),
 ('murd', 188),
 ('vasem', 179),
 ('post', 173),
 ('4', 169),
 ('keskmine', 158),
 ('opereerima', 157),
 ('silm', 155),
 ('mõlema', 154),
 ('sündroom', 153),
 ('et', 149),
 ('jalg', 147),
 ('kerge', 141),
 ('staadium', 141),
 ('pahaloomuline', 140),
 ('äge', 135),
 ('põlveliiges', 129),
 ('IV', 128),
 ('5', 127),
 ('kõrge', 124),
 ('põlv', 118),
 ('ravi', 118),
 ('piirkond', 117),
 ('puusaliiges', 114),
 ('järgne', 109),
 ('käsi', 109),
 ('artroos', 103),
 ('ajuarter', 103),
 ('organkahjustus', 100),
 ('esmane', 99)]

In [8]:
pd.DataFrame(Counter(lemmas).most_common()).to_csv("temp/lemmas.csv")

In [9]:
Counter(words).most_common(50)

[('-', 1223),
 ('ja', 1026),
 ('hüpertooniatõbi', 365),
 ('Südamekahjustusega', 362),
 ('2', 317),
 ('III', 314),
 ('3', 267),
 ('või', 236),
 ('paremal', 234),
 ('aste', 225),
 ('ilma', 223),
 ('1', 199),
 ('täpsustamata', 193),
 ('südamepuudulikkuseta', 193),
 ('kasvaja', 191),
 ('risk', 186),
 ('vasakul', 185),
 ('4', 169),
 ('post', 167),
 ('südamepuudulikkusega', 163),
 ('Parema', 160),
 ('tegevus', 157),
 ('murd', 157),
 ('täpsustatud', 157),
 ('muu', 156),
 ('muud', 138),
 ('et', 137),
 ('kahtlus', 135),
 ('parem', 135),
 ('5', 127),
 ('IV', 123),
 ('sündroom', 123),
 ('Paremal', 121),
 ('vasak', 121),
 ('kõrge', 120),
 ('Vasaku', 117),
 ('Parem', 117),
 ('opereeritud', 116),
 ('Kahtlus', 113),
 ('Krooniline', 112),
 ('Muu', 108),
 ('muude', 106),
 ('Täpsustamata', 105),
 ('pahaloomuline', 105),
 ('staadium', 103),
 ('artroos', 99),
 ('Muud', 98),
 ('Vasak', 96),
 ('tõttu', 95),
 ('organkahjustus', 92)]

In [10]:
pd.DataFrame(Counter(words).most_common()).to_csv("temp/words.csv")

In [11]:
Counter(unknowns).most_common(50)

[('kongestiivse', 361),
 ('Grav', 190),
 ('hebd', 144),
 ('oidiit', 88),
 ('Ruptura', 76),
 ('bilateralis', 73),
 ('Status', 63),
 ('otoksikoos', 61),
 ('bilat', 61),
 ('uteri', 61),
 ('radikulopaatia', 59),
 ('Gonarthrosis', 58),
 ('radikulopaatiaga', 56),
 ('sanata', 53),
 ('genus', 53),
 ('intraokulaarse', 50),
 ('coli', 48),
 ('seniilkatarakt', 47),
 ('Laesio', 46),
 ('Arthrosis', 44),
 ('hemiparees', 44),
 ('Mitraal', 42),
 ('mammae', 40),
 ('grav', 40),
 ('genu', 39),
 ('Cysta', 39),
 ('partus', 39),
 ('labii', 37),
 ('cruris', 37),
 ('Aterosklerootiline', 35),
 ('dextri', 35),
 ('Insuff', 34),
 ('Fractura', 33),
 ('ovarii', 33),
 ('prolaps', 32),
 ('haigussesundid', 32),
 ('hüpotüreoos', 30),
 ('Paroksüsmaalne', 30),
 ('stent', 30),
 ('Coxarthrosis', 30),
 ('Carcinoma', 30),
 ('paroksüsmaalne', 29),
 ('vaginae', 29),
 ('menisci', 29),
 ('cordis', 29),
 ('operata', 29),
 ('medialis', 28),
 ('Cicatrix', 28),
 ('femoris', 28),
 ('osteo', 28)]

In [12]:
pd.DataFrame(Counter(unknowns).most_common()).to_csv("temp/unknowns.csv")

In [13]:
other_other[:50]

[['Muu spondüloos', 'M00-M99', '186'],
 ['Täielik või täpsustamata, tüsistusteta', 'O00-O99', '113'],
 ['Somatoformne vegetatiivne düsfunktsioon', 'F00-F99', '108'],
 ['Kodu, sportlik tegevus', 'None', '95'],
 ['Kõigi paikmete hemangioom', 'C00-D48', '81'],
 ['-oluline käitumishäire', 'F00-F99', '80'],
 ['Kodu, muu täpsustatud tegevusega', 'None', '69'],
 ['Kerge depressioon', 'F00-F99', '55'],
 ['Keskmise raskusega', 'G00-G99', '47'],
 ['Nimme-ristluu piirkond', 'None', '40'],
 ['Krooniline kõripõletik', 'J00-J99', '40'],
 ['Nimmelülimurd, kinnine', 'S00-T98', '37'],
 ['Mittetäielik, tüsistusteta', 'O00-O99', '36'],
 ['Preproliferatiivne,osaliselt laserdatud', 'None', '35'],
 ['Tänav ja [maan]tee', 'V01-Y98', '34'],
 ['Ebastabiilne isiksus', 'F00-F99', '32'],
 ['Vasak õlg', 'M00-M99', '32'],
 ['Põlveõndla limapaunatsüst [baker]', 'M00-M99', '30'],
 ['Autoimmuunne türeoidiit', 'E00-E90', '30'],
 ['Anaemia gravidarum', 'O00-O99', '30'],
 ['Orgaanilised meeleoluhäired, orgaaniline de', '

In [14]:
pd.DataFrame(other_other).to_csv("temp/other_other.csv")

In [15]:
repetition[:50]

[['Kliiniliselt\nkliiniliselt', 'A00-B99', '36'],
 ['Diffuusne\ndiffuusne', 'L00-L99', '22'],
 ['Näol\nnäol', 'L00-L99', '18'],
 ['Autoimmuunne tür[e]oidiit; hüpotüreoos\nhüpotüreoos', 'E00-E90', '16'],
 ['Raskekujuline\nraskekujuline', 'G00-G99', '14'],
 ['Oc.utr.\noc.utr.', 'Z00-Z99', '12'],
 ['Köhavariantne\nköhavariantne', 'J00-J99', '10'],
 ['Sin.\nsin.', 'H60-H95', '9'],
 ['Kahtlus\nkahtlus', 'J00-J99', '9'],
 ['Kahtlus\nkahtlus', 'G00-G99', '7'],
 ['Oc.utr.\noc.utr.', 'H00-H59', '7'],
 ['L-Thyroxinravil\nL-Thyroxinravil', 'E00-E90', '6'],
 ['Silmasisese e intraokulaarse läätse olemasolu; OD\nOD', 'Z00-Z99', '6'],
 ['Od\nod', 'H00-H59', '5'],
 ['Opereeritud\nopereeritud', 'I00-I99', '5'],
 ['Tablettravil\ntablettravil', 'E00-E90', '5'],
 ['Kätel\nkätel', 'L00-L99', '5'],
 ['2006a\n2006a', 'I00-I99', '5'],
 ['Pedum\npedum', 'L00-L99', '5'],
 ['E pk pk pT2N0M0G3  I st.  IIb--IIIkl. gr- 1999.a.', 'C00-D48', '4'],
 ['Peanahk\npeanahk', 'L00-L99', '4'],
 ['Algav seniilkatarakt; OU\nOU

In [16]:
pd.DataFrame(repetition).to_csv("temp/repetition.csv")

In [17]:
capital[:50]

[['F70.1   olulise käitumishäirega', 'None', '28'],
 ['. Hemokromatoos (homosügoot C282Y)', 'E00-E90', '26'],
 ['Rauaainevahetuse häired.  (homosügoot C282Y)', 'E00-E90', '26'],
 ['I11.9  Südamekahjustusega hüpertooniatõbi. Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta',
  'I00-I99',
  '21'],
 ['Raskekujuline, kasutab CPAP aparaati\nraskekujuline, kasutab CPAP aparaati',
  'G00-G99',
  '19'],
 ['G25.0  Muud ekstrapüramidaal- ja liigutushäired.', 'G00-G99', '14'],
 ['G25.0  . Essentsiaalne treemor', 'G00-G99', '14'],
 ['IVF. Gravid in hebd 6', 'Z00-Z99', '12'],
 ['Th 6-11 paravertebraalne perifeerne primitiivne neuroektodermaalne tuumor/ekstraossaalne Ewingi sarkoom ; c T2 N0 M1B IVBst\nTh 6-11 paravertebraalne perifeerne primitiivne neuroektodermaalne tuumor/ekstraossaalne Ewingi sarkoom',
  'C00-D48',
  '10'],
 ['Raskekujuline, kasutab CPAP aparaati', 'G00-G99', '10'],
 ['F71.1   oluliste käitumishäiretega', 'None', '10'],
 ['HPV 16', 'A00-B99', '9'],
 ['

In [18]:
pd.DataFrame(capital).to_csv("temp/capital.csv")

In [19]:
arabic[:50]

[['Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta; Riskiaste 2',
  'I00-I99',
  '65'],
 ['Riskiaste 2', 'I00-I99', '58'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega; Riskiaste 2',
  'I00-I99',
  '37'],
 ['Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta; Riskiaste 3',
  'I00-I99',
  '34'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega; Riskiaste 3',
  'I00-I99',
  '31'],
 ['F70.1   olulise käitumishäirega', 'None', '28'],
 ['. Hemokromatoos (homosügoot C282Y)', 'E00-E90', '26'],
 ['Rauaainevahetuse häired.  (homosügoot C282Y)', 'E00-E90', '26'],
 ['Glomeerulite haigusseisundid melliitdiabeedi korral (e10-e14 ühise neljanda tärgiga.2)',
  'N00-N99',
  '24'],
 ['I11.9  Südamekahjustusega hüpertooniatõbi. Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta',
  'I00-I99',
  '21'],
 ['Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikk

In [20]:
pd.DataFrame(arabic).to_csv("temp/arabic.csv")

In [21]:
roman[:50]

[['Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta II',
  'I00-I99',
  '111'],
 ['NYHA II', 'I00-I99', '66'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega II aste (organkahjustus III, ülikõrge kardiovaskulaarne risk)',
  'I00-I99',
  '45'],
 ['III st.', 'I00-I99', '42'],
 ['II NYHA', 'I00-I99', '42'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega I aste (organkahjustus III, ülikõrge kardiovaskulaarne risk)',
  'I00-I99',
  '37'],
 ['I pk', 'C00-D48', '34'],
 ['Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta,II',
  'I00-I99',
  '33'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega II',
  'I00-I99',
  '31'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega I aste (organkahjustus II, kõrge kardiovaskulaarne risk)',
  'I00-I99',
  '31'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega III',
  'I00-I99'

In [22]:
pd.DataFrame(roman).to_csv("temp/roman.csv")

In [23]:
olemasolu

[['Ou , silmasisese e intraokulaarse läätse olemasolu', 'Z00-Z99', '80'],
 ['Od , silmasisese e intraokulaarse läätse olemasolu', 'Z00-Z99', '18'],
 ['Od  , silmasisese e intraokulaarse läätse olemasolu', 'Z00-Z99', '13'],
 ['Os , silmasisese e intraokulaarse läätse olemasolu', 'Z00-Z99', '10'],
 ['Silmasisese e intraokulaarse läätse olemasolu ou', 'Z00-Z99', '7'],
 ['Silmasisese e intraokulaarse läätse olemasolu; OD\nOD', 'Z00-Z99', '6'],
 ['Os  , silmasisese e intraokulaarse läätse olemasolu', 'Z00-Z99', '5'],
 ['Ou  , silmasisese e intraokulaarse läätse olemasolu', 'Z00-Z99', '5'],
 ['Silmasisese e intraokulaarse läätse olemasolu OD', 'Z00-Z99', '5'],
 ['Silmasisese e intraokulaarse läätse olemasolu os', 'Z00-Z99', '5'],
 ['Silmasisese e intraokulaarse läätse olemasolu; mõlemad silmad\nmõlemad silmad',
  'Z00-Z99',
  '4'],
 ['Silmasisese e intraokulaarse läätse olemasolu,os', 'Z00-Z99', '4'],
 ['Silmasisese e intraokulaarse läätse olemasolu OS', 'Z00-Z99', '3'],
 ['Silmasisese e int

In [24]:
pd.DataFrame(olemasolu).to_csv("temp/olemasolu.csv")

In [25]:
kasutab[:50]

[['Raskekujuline, kasutab CPAP aparaati\nraskekujuline, kasutab CPAP aparaati',
  'G00-G99',
  '19'],
 ['Raskekujuline, kasutab CPAP aparaati', 'G00-G99', '10'],
 ['Uinumise ja une säilitamise häired [insomniad]\nkasutab unerohtu',
  'G00-G99',
  '4'],
 ['Kasutab kuuldeaparaati', 'H60-H95', '2'],
 ['Antikoagulantide (käesolev) pikaajaline kasutamine anamneesis\nalates 2010a varfariiniga antikoagulantravil',
  'Z00-Z99',
  '2'],
 ['NSAID kasutamine', 'K00-K93', '2'],
 ['Antikoagulantide (käesolev) pikaajaline kasutamine anamneesis; Xarelto; KATE sept.2014',
  'Z00-Z99',
  '1'],
 ['Pillide valest kasutamisest tingitud', 'N00-N99', '1'],
 ['Pillide kasutamise järgselt', 'N00-N99', '1'],
 ['Cordaroni kasutamise foonil', 'E00-E90', '1'],
 ['Muude ravimite (käesolev) pikaajaline kasutamine anamneesis - ASPIRIIN',
  'Z00-Z99',
  '1'],
 ['Antikoagulantide (käesolev) pikaajaline kasutamine anamneesis\nanamneesis KATE',
  'Z00-Z99',
  '1'],
 ['Antikoagulantide (käesolev) pikaajaline kasutamine a

In [26]:
pd.DataFrame(kasutab).to_csv("temp/kasutab.csv")

In [27]:
nyha[:50]

[['NYHA II', 'I00-I99', '66'],
 ['II NYHA', 'I00-I99', '42'],
 ['NYHA III', 'I00-I99', '26'],
 ['NYHA II staadium', 'I00-I99', '21'],
 ['III st. NYHA', 'I00-I99', '14'],
 ['II aste NYHA j.', 'I00-I99', '10'],
 ['III NYHA', 'I00-I99', '9'],
 ['II st. NYHA', 'I00-I99', '9'],
 ['III aste NYHA j.', 'I00-I99', '8'],
 ['NYHA I-II staadium', 'I00-I99', '8'],
 ['NYHA II-III', 'I00-I99', '8'],
 ['NYHA III st', 'I00-I99', '7'],
 ['NYHA II st.', 'I00-I99', '7'],
 ['IV NYHA', 'I00-I99', '6'],
 ['I NYHA', 'I00-I99', '6'],
 ['NYHA IV', 'I00-I99', '6'],
 ['II aste NYHA j', 'I00-I99', '6'],
 ['II-III st. NYHA', 'I00-I99', '6'],
 ['NYHA II-III st', 'I00-I99', '6'],
 ['II NYHA j.', 'I00-I99', '6'],
 ['NYHA II\nNYHA II', 'I00-I99', '6'],
 ['NYHA II-III staadium', 'I00-I99', '6'],
 ['III st. NYHA j.', 'I00-I99', '5'],
 ['II-III aste NYHA j.', 'I00-I99', '5'],
 ['NYHA II st', 'I00-I99', '5'],
 ['IV aste NYHA j.', 'I00-I99', '4'],
 ['NYHA IV staadium', 'I00-I99', '4'],
 ['- III st. NYHA', 'I00-I99', '4'],
 

In [28]:
pd.DataFrame(nyha).to_csv("temp/nyha.csv")

In [29]:
risk[:50]

[['Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta; Riskiaste 2',
  'I00-I99',
  '65'],
 ['Riskiaste 2', 'I00-I99', '58'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega II aste (organkahjustus III, ülikõrge kardiovaskulaarne risk)',
  'I00-I99',
  '45'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega; Riskiaste 2',
  'I00-I99',
  '37'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega I aste (organkahjustus III, ülikõrge kardiovaskulaarne risk)',
  'I00-I99',
  '37'],
 ['Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta; Riskiaste 3',
  'I00-I99',
  '34'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega I aste (organkahjustus II, kõrge kardiovaskulaarne risk)',
  'I00-I99',
  '31'],
 ['Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega; Riskiaste 3',
  'I00-I99',
  '31'],
 ['Südamekahjustusega hüpertooniatõbi

In [30]:
pd.DataFrame(risk).to_csv("temp/risk.csv")

In [31]:
longtext[:50]

[['Kodu, puhke- ja vaba aja tegevus', 'None', '994'],
 ['Suulae- [kurgu-] ja [nina-]neelumandlite kroonilised haigused, krooniline mandlipõletik e tonsilliit',
  'J00-J99',
  '528'],
 ['Tür[e]oidiit e kilpnäärmepõletik, autoimmuunne tür[e]oidiit',
  'E00-E90',
  '489'],
 ['Tänav ja [maan]tee, puhke- ja vaba aja tegevus', 'None', '379'],
 ['Dermatofütoos e nahaseenhaigustu[mu]s, küünte nahaseenhaigustus e tiinea',
  'A00-B99',
  '250'],
 ['Tür[e]otoksikoos [hüpertüreoos], tür[e]otoksikoos difuusse struumaga',
  'E00-E90',
  '219'],
 ['Kodus puhke- või vabaaja tegevuse juures', 'None', '211'],
 ['Kodarluu ala- [distaalse] otsa murd, kinnine', 'S00-T98', '182'],
 ['Viirustüükad [soolatüükad] e harilikud tüükad', 'A00-B99', '127'],
 ['Kodu, muu töö ja tegevus', 'None', '114'],
 ['Tür[e]otoksikoos [hüpertüreoos], tür[e]otoksikoos toksilise hulgisõlmelise struumaga',
  'E00-E90',
  '111'],
 ['Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta II',
  'I00-I99',
  '111

In [32]:
pd.DataFrame(longtext).to_csv("temp/longtext.csv")